In [2]:
!pip install -U sentence-transformers
!pip install pinecone-client
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 112.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d531570ae5dfa749fd4b1be6c49da2ae41e7bb3a8ba67211acfc422d17a829c2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.2 MB

In [8]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from ast import literal_eval

In [10]:
df = pd.read_csv('/content/25k_movie_imdb.csv')

In [11]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/


In [12]:
def concatenar_lista(lista):
  lista = literal_eval(lista)
  return ' '.join(lista)

In [13]:
def string_to_list(lista):
  lista = literal_eval(lista)
  return lista


In [14]:
df.columns

Index(['movie title', 'Run Time', 'Rating', 'User Rating', 'Generes',
       'Overview', 'Plot Kyeword', 'Director', 'Top 5 Casts', 'Writer', 'year',
       'path'],
      dtype='object')

In [15]:
df = df.fillna(' ')
df['Keywords'] = df['Plot Kyeword'].apply(concatenar_lista)
df['Stars'] = df['Top 5 Casts'].apply(concatenar_lista)
df['Generes'] = df['Generes'].apply(string_to_list)
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(0).astype('float')

In [16]:
unique_generes = df['Generes'].explode().unique()
unique_generes

array(['Action', 'Drama', 'Adventure', 'Sci-Fi', 'Animation', 'Crime',
       'Comedy', 'Thriller', 'Fantasy', 'Horror', 'History', 'Mystery',
       'Biography', 'War', 'Western', 'Sport', 'Family', 'Romance',
       'Music', 'Musical', 'Film-Noir', 'Game-Show', 'Adult',
       'Reality-TV'], dtype=object)

In [17]:
df.drop(['Plot Kyeword', 'Top 5 Casts'], axis=1, inplace=True)

In [18]:
df['text'] = df.apply(lambda x : str(x['Overview']) + ' ' + str(x['Keywords']) + ' ' + str(x['Stars']), axis=1)

In [19]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Stars,text
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"[Action, Drama]",After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,-2022,/title/tt1745960/,fighter jet sequel u.s. navy fighter aircraft ...,Jack Epps Jr. Peter Craig Tom Cruise Jennifer ...,After more than thirty years of service as one...
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"[Action, Adventure, Sci-Fi]",Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,-2022,/title/tt8041270/,dinosaur jurassic park tyrannosaurus rex veloc...,Colin Trevorrow Derek Connolly Chris Pratt Bry...,Four years after the destruction of Isla Nubla...
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"[Action, Drama]",As students at the United States Navy's elite ...,Tony Scott,Jim Cash,-1986,/title/tt0092099/,pilot male camaraderie u.s. navy grumman f 14 ...,Jack Epps Jr. Ehud Yonay Tom Cruise Tim Robbin...,As students at the United States Navy's elite ...


In [20]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [21]:
embeddings = model.encode(df['text'], batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/382 [00:00<?, ?it/s]

In [22]:
df['embeddings'] = embeddings.tolist()
df['ids'] = df.index
df['ids'] = df['ids'].astype('str')


In [23]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Stars,text,embeddings,ids
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"[Action, Drama]",After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,-2022,/title/tt1745960/,fighter jet sequel u.s. navy fighter aircraft ...,Jack Epps Jr. Peter Craig Tom Cruise Jennifer ...,After more than thirty years of service as one...,"[-0.07095596939325333, -0.009481017477810383, ...",0
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"[Action, Adventure, Sci-Fi]",Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,-2022,/title/tt8041270/,dinosaur jurassic park tyrannosaurus rex veloc...,Colin Trevorrow Derek Connolly Chris Pratt Bry...,Four years after the destruction of Isla Nubla...,"[-0.02536219358444214, -0.06149570643901825, 0...",1
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"[Action, Drama]",As students at the United States Navy's elite ...,Tony Scott,Jim Cash,-1986,/title/tt0092099/,pilot male camaraderie u.s. navy grumman f 14 ...,Jack Epps Jr. Ehud Yonay Tom Cruise Tim Robbin...,As students at the United States Navy's elite ...,"[-0.007393533363938332, 0.02564958482980728, -...",2


In [3]:
import pinecone
from getpass import getpass

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
pinecone_api = getpass('Pinecone API Key: ')


Pinecone API Key: ··········


In [5]:
pinecone.init(api_key=pinecone_api, environment='us-west1-gcp')

In [26]:
dimension_embedding = len(df['embeddings'][0])

index_name = 'movies-embeddings'
all_index = pinecone.list_indexes()
if index_name in all_index:
  index = pinecone.Index(index_name)
else:
  pinecone.create_index(index_name, dimension=dimension_embedding, metric='cosine')
  index = pinecone.Index(index_name)


In [27]:
index

#### Ingest Data

In [28]:
from tqdm.auto import tqdm

# We will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
  i_end = min(i + batch_size, len(df))
  # extract batch
  batch = df[i:i_end]
  ids = batch['ids']
  emb = batch['embeddings']
  metadata = batch.drop(['ids', 'embeddings', 'text', 'path'], axis=1).to_dict('records')
  # add all to upsert list
  to_upsert = list(zip(ids, emb, metadata))
  _ = index.upsert(vectors=to_upsert)

index.describe_index_stats()

  0%|          | 0/382 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 24402}},
 'total_vector_count': 24402}

In [32]:
def search(query, genre, rating, top_k):
  query_vector = model.encode(query).tolist()

  if rating:
    filter_rating = rating
  else:
    filter_rating = 0

  if genre:
    conditions = {
        'Generes': {
            "$in": [genre]
        },
        'Rating': {
            "$gte": filter_rating
        }
    }
  else:
    conditions = {
        'Rating': {
            "$gte": filter_rating
        }
    }


  responses = index.query(
      query_vector,
      top_k=top_k,
      include_metadata=True,
      filter=conditions)

  responses_data = []
  for response in responses['matches']:
    responses_data.append(
        {
            'Title': response['metadata']['movie title'],
            'Overview': response['metadata']['Overview'],
            'Director': response['metadata']['Director'],
            'Generes': response['metadata']['Generes'],
            'Year': response['metadata']['year'],
            'Rating': response['metadata']['Rating'],
            'Score': response['score'],

        }
    )

  df = pd.DataFrame(responses_data)
  return df



In [33]:
search('A time travel adventure', None, None, 5)

,Title,Overview,Director,Generes,Year,Rating,Score
0,Timeline,A group of archaeologists become trapped in th...,Richard Donner,"[Action, Adventure, Sci-Fi]",-2003,5.6,0.670312
1,The Exotic Time Machine,A time machine transports two curious pleasure...,Felicia Sinclair,[Sci-Fi],-1998,3.6,0.668018
2,Norman,A time traveler and his A.I. companion. Norman...,Joel Guelzo,"[Adventure, Sci-Fi, Thriller]",(IV) (2019),3.6,0.663255
3,Zärtliche Chaoten II,3 chaotic unlucky fellows travel back in time ...,Holm Dressler,"[Comedy, Sci-Fi, Romance]",-1988,4.3,0.637484
4,A Sound of Thunder,"A single mistake in the past, by a time travel...",Peter Hyams,"[Action, Adventure, Horror]",-2005,4.2,0.637212


In [34]:
import gradio as gr

# Define possible genres
genres = unique_generes.tolist()

# Create Interface
iface = gr.Interface(
    fn=search,
    inputs=[
        gr.Textbox(lines=5, placeholder="Escribe aqui tu consulta... ", label="Consulta"),
        gr.Dropdown(choices=genres, label="Género de la pelîcula"),
        gr.Slider(minimum=1, maximum=10, value=5, label="Puntuacion minima"),
        gr.Number(minimu=1, maximum=10, value=3, label="Número de resultados")
    ],
    outputs=gr.Dataframe(type="pandas", label="Resultados"),
    title = "Find your movie 🎞️",
    description = "Best place to find your movie 🍿"
)

iface.launch()


<ipython-input-34-7fb112cef7bc>:13: GradioUnusedKwargWarning: You have unused kwarg parameters in Number, please remove them: {'minimu': 1}
  gr.Number(minimu=1, maximum=10, value=3, label="Número de resultados")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ed553dd64b7da48fa8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
